# LFM2-1.2B v6.1 Curriculum Learning (Phase 2)

**Phase 2: High Quality Curriculum Learning**
1. **Base Model**: `gyung/lfm2-1.2b-koen-mt-v6-sft-200k` (Phase 1 완료, Step 12504, Loss 1.3220)
2. **Dataset**: `lfm2_koen_v6_sft_200k_filtered.jsonl` (CometKiwi > 0.85, 80,611개)
3. **Training**: 1 epoch on high quality data with lower learning rate (5e-6)
4. **Output**: `gyung/lfm2-1.2b-koen-mt-v6.1-curriculum`

---

### 🌟 Kaggle 설정 (필수)
1. **Accelerator**: **GPU T4 x 2**
2. **Internet**: **On**
3. **Secrets**: `HF_TOKEN` 설정
4. **Dataset**: `lfm2_koen_v6_sft_200k_filtered.jsonl` 업로드 필요

### ⏱️ 예상 시간
- ~5000 steps (~4-5시간)


In [ ]:
# 1. 캐시 디렉토리 설정 (/tmp 사용 - 디스크 절약)
import os
os.environ["HF_HOME"] = "/tmp/hf_cache"
os.environ["HF_DATASETS_CACHE"] = "/tmp/hf_cache/datasets"
os.environ["TRANSFORMERS_CACHE"] = "/tmp/hf_cache/transformers"

# 2. 필수 패키지 설치
!pip install -q transformers==4.54.0 trl>=0.18.2 peft>=0.15.2 datasets accelerate bitsandbytes huggingface_hub protobuf==3.20.3

from kaggle_secrets import UserSecretsClient

# 3. Hugging Face 토큰
try:
    user_secrets = UserSecretsClient()
    os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_TOKEN")
    print("✅ HF_TOKEN 환경 변수 설정 완료!")
except Exception as e:
    print(f"⚠️ 토큰 설정 실패: {e}")

In [ ]:
%%writefile train_phase2_curriculum.py
import os
import gc
import shutil
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainerCallback
from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer, SFTConfig, setup_chat_format
from huggingface_hub import login, HfApi

# ===========================================
# Custom Callback: Explicit Commit Message
# ===========================================
class CurriculumCallback(TrainerCallback):
    """명시적 commit message로 업로드 (DDP: rank 0만)"""
    def __init__(self, hf_token, hub_model_id):
        self.hf_token = hf_token
        self.hub_model_id = hub_model_id
        self.api = HfApi(token=hf_token)
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        """로그 출력"""
        if logs and 'loss' in logs:
            print(f"📊 [Step {state.global_step}] loss={logs.get('loss', 'N/A'):.4f}")
    
    def on_save(self, args, state, control, **kwargs):
        """체크포인트 저장 시 업로드 (DDP: rank 0만)"""
        local_rank = getattr(args, 'local_rank', -1)
        if local_rank not in [-1, 0]:
            return control
        
        checkpoint_dir = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
        
        if os.path.exists(checkpoint_dir):
            loss_val = 'N/A'
            if state.log_history:
                for log in reversed(state.log_history):
                    if 'loss' in log:
                        loss_val = f"{log['loss']:.4f}"
                        break
            commit_msg = f"Step {state.global_step} checkpoint (loss: {loss_val})"
            print(f"\n🚀 Uploading with commit: '{commit_msg}'")
            try:
                self.api.upload_folder(
                    folder_path=checkpoint_dir,
                    repo_id=self.hub_model_id,
                    commit_message=commit_msg,
                    repo_type="model"
                )
                print(f"✅ Uploaded Step {state.global_step}")
            except Exception as e:
                print(f"⚠️ Upload failed: {e}")
        return control

def find_dataset(target_files):
    for root, dirs, files in os.walk("/kaggle/input"):
        for target in target_files:
            if target in files:
                return os.path.join(root, target)
    return None

def create_mt_dataset(file_path):
    """데이터셋 생성 (양방향 번역)"""
    print(f"🔄 Loading dataset from {file_path}...")
    base_dataset = load_dataset("json", data_files=file_path, split="train")
    
    def format_en2ko(row):
        return {"messages": [{"role":"system","content":"Translate to Korean."}, 
                            {"role":"user","content":row['input']}, 
                            {"role":"assistant","content":row['output']}]}
    def format_ko2en(row):
        return {"messages": [{"role":"system","content":"Translate to English."}, 
                            {"role":"user","content":row['output']}, 
                            {"role":"assistant","content":row['input']}]}
    
    ds_en2ko = base_dataset.map(format_en2ko, remove_columns=base_dataset.column_names, num_proc=4)
    ds_ko2en = base_dataset.map(format_ko2en, remove_columns=base_dataset.column_names, num_proc=4)
    
    full_dataset = concatenate_datasets([ds_en2ko, ds_ko2en])
    full_dataset = full_dataset.shuffle(seed=42)
    
    print(f"📊 Total Dataset Size: {len(full_dataset):,}")
    return full_dataset

def cleanup_gpu():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    print("🧹 GPU Memory Cleaned")

def cleanup_checkpoints(output_dir):
    if os.path.exists(output_dir):
        print(f"🧹 Cleaning up: {output_dir}")
        shutil.rmtree(output_dir, ignore_errors=True)
        print("✅ Disk space freed")

# ==========================================
# Phase 2: Curriculum Learning (80k HQ)
# ==========================================
def train_phase2_curriculum(hf_token):
    print("\n" + "="*60)
    print("🎓 PHASE 2: High Quality Curriculum Learning (1k HQ)")
    print("="*60 + "\n")
    
    PHASE1_MODEL_ID = "gyung/lfm2-1.2b-koen-mt-v6-sft-200k"  # Phase 1 완료 모델
    DATASET_REPO = "gyung/lfm2-koen-samples"
    DATASET_CONFIG = "manual_1000_sft"    # 1k 고품질 데이터
    OUTPUT_DIR = "/kaggle/working/phase2_curriculum"
    HUB_MODEL_ID = "gyung/lfm2-1.2b-koen-mt-v6.1-curriculum"
    
    dataset_path = find_dataset([DATASET_FILE])
    if not dataset_path:
        print(f"❌ Dataset not found: {DATASET_FILE}")
        return None
    
    # 1. Phase 1 모델 로드
    print(f"📦 Loading Phase 1 Model: {PHASE1_MODEL_ID}")
    tokenizer = AutoTokenizer.from_pretrained(PHASE1_MODEL_ID)
    model = AutoModelForCausalLM.from_pretrained(PHASE1_MODEL_ID, torch_dtype="float16")
    
    if tokenizer.chat_template is None:
        model, tokenizer = setup_chat_format(model, tokenizer)
    
    # 2. 고품질 데이터셋 준비
    print(f"📚 Loading: {DATASET_REPO} ({DATASET_CONFIG})")
                train_dataset = load_dataset(DATASET_REPO, DATASET_CONFIG, split="train")
    
    # 3. 학습 설정 (낮은 LR로 Fine-tuning)
    sft_config = SFTConfig(
        output_dir=OUTPUT_DIR,
        hub_model_id=HUB_MODEL_ID,
        hub_token=hf_token,
        push_to_hub=False,  # Callback에서 처리
        num_train_epochs=1,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        gradient_checkpointing=True,
        optim="paged_adamw_8bit",
        fp16=False,
        bf16=False,
        learning_rate=5e-6,  # 낮은 LR
        lr_scheduler_type="cosine",
        warmup_ratio=0.05,
        logging_steps=250,
        save_strategy="steps",
        save_steps=1000,
        save_total_limit=2,
        dataset_text_field="messages",
        packing=False,
        report_to="none",
        ddp_find_unused_parameters=False,
    )
    
    # 4. Callback 추가
    curriculum_callback = CurriculumCallback(
        hf_token=hf_token,
        hub_model_id=HUB_MODEL_ID
    )
    
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        args=sft_config,
        processing_class=tokenizer,
        callbacks=[curriculum_callback],
    )
    
    print("🚀 Starting Phase 2 Training...")
    print(f"   Learning Rate: 5e-6 (Lower for curriculum)")
    print(f"   Dataset: {len(train_dataset):,} samples")
    trainer.train()
    
    # 5. 최종 모델 저장 및 업로드
    print("💾 Phase 2: Final Saving & Uploading...")
    trainer.save_model(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    
    # DDP rank 0만 최종 업로드
    local_rank = int(os.environ.get('LOCAL_RANK', -1))
    if local_rank in [-1, 0]:
        api = HfApi(token=hf_token)
        final_step = trainer.state.global_step
        try:
            # README.md 생성 (업로드 오류 방지)
            readme_path = os.path.join(OUTPUT_DIR, "README.md")
            with open(readme_path, "w") as f:
                f.write(f"# LFM2-1.2B-KoEn-MT v6.1 Curriculum\\n\\n")
                f.write(f"Fine-tuned on 80k high-quality EN-KO translation pairs.\\n\\n")
                f.write(f"Base: gyung/lfm2-1.2b-koen-mt-v6-sft-200k\\n")
                f.write(f"Final Step: {final_step}\\n")
            api.upload_folder(
                folder_path=OUTPUT_DIR,
                repo_id=HUB_MODEL_ID,
                commit_message=f"v6.1 Curriculum Complete - Step {final_step} (80k HQ Final)",
                repo_type="model"
            )
            print(f"✅ Phase 2 Complete! Model: {HUB_MODEL_ID}")
        except Exception as e:
            print(f"⚠️ Final upload failed: {e}")
    
    # 메모리 정리
    del trainer, model
    cleanup_gpu()
    
    return HUB_MODEL_ID

# ==========================================
# Main Execution
# ==========================================
if __name__ == "__main__":
    print("="*60)
    print("🎓 LFM2-v6.1 Curriculum Learning (Phase 2)")
    print("="*60)
    
    hf_token = os.environ.get("HF_TOKEN")
    if not hf_token:
        raise ValueError("❌ HF_TOKEN not found!")
    
    login(token=hf_token)
    print("✅ Logged in to Hugging Face Hub")
    
    # GPU 정보
    if torch.cuda.is_available():
        print(f"🖥️ GPU: {torch.cuda.get_device_name(0)}")
        print(f"🖥️ GPU Count: {torch.cuda.device_count()}")
    
    # Phase 2 실행
    final_model = train_phase2_curriculum(hf_token)
    
    if final_model:
        print("\n" + "="*60)
        print("🎉 CURRICULUM LEARNING COMPLETE!")
        print(f"📦 Final Model: {final_model}")
        print("="*60)
    else:
        print("\n❌ Training failed.")

In [ ]:
# GPU 메모리 최적화
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# DDP로 학습 실행
!accelerate launch --multi_gpu --num_processes=2 train_phase2_curriculum.py

In [ ]:
# 🧪 학습된 모델 테스트
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "gyung/lfm2-1.2b-koen-mt-v6.1-curriculum"

print(f"🔹 Loading: {MODEL_ID}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype="float16", device_map="auto")

def translate(text, to="ko"):
    system = "Translate to Korean." if to == "ko" else "Translate to English."
    messages = [{"role": "system", "content": system}, {"role": "user", "content": text}]
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.3,
            min_p=0.15,
            repetition_penalty=1.05,
        )
    return tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

# 테스트
tests = [
    "Please fix this error.",
    "The model is working correctly now.",
    "Time flies like an arrow.",
]

for text in tests:
    result = translate(text, to="ko")
    print(f"📝 EN: {text}")
    print(f"🇰🇷 KO: {result}")
    print("-" * 40)